In [ ]:
import pickle
import plotly.express as px
import numpy as np
import cv2
import json
from tqdm.notebook import tqdm
import einops
import matplotlib.pyplot as plt
from pathlib import Path
import os
from skimage.measure import find_contours
from visualization_utils.visualization import draw_on_image

from blender_cv_utils.projection_camera import apply_object_matrix_world, project_points

In [ ]:
path = 'C:/Users/ivand/Desktop/output_temp/default/'

global_params = json.load(open(os.path.join(path, 'global_params.json'), 'r'))

all_data = []
for n, annot_path in tqdm(enumerate(sorted(Path(path).rglob('annots_frame*.json')))):
     im = cv2.imread(os.path.join(path, f'Camera_{n+1:04}.png'))
     data = json.load(
         open(os.path.join(path, f'annots_frame{n+1:04}.json'), 'r'))
     data['Camera'].update(global_params)

     objects_data = {}
     for index, i in enumerate(['Armature', 'Armature.001']):
          skeleton = []
          for name, bone in data[i]['pose'].items():
              if name in ['Bone.012.R.001', 'Bone.012.L.001', 'Bone.008.R.001', 'Bone.008.L.001', 'Bone.020']:  # IK bones
                  continue
              points = np.stack([np.array(bone['head']), np.array(bone['tail'])])
              points = apply_object_matrix_world(points, data[i]['matrix_world'])
              points = project_points(points, **data['Camera']).numpy()
              skeleton.append(points)

          mesh = data[data[i]['childrens'][0]]
          mask = plt.imread(os.path.join(path, mesh["name"], f'Camera_{n+1:04}.png'))[:, :, 3]
          xraymask = plt.imread(os.path.join(path, mesh["name"], f'xray_Camera_{n+1:04}.png'))[:, :, 3]

          im = draw_on_image(im, mask)
       
          contours = find_contours(mask.T, 0.5)
          contours = [np.array(i).round().astype(np.int32) for i in contours]
          contours = np.concatenate(contours)
          xmin, xmax = np.min(contours[:, 0]), np.max(contours[:, 0])
          ymin, ymax = np.min(contours[:, 1]), np.max(contours[:, 1])
          objects_data[i] = [
              (xmin, ymin, xmax, ymax),
              mask[ymin:ymax, xmin:xmax],
              np.stack(skeleton)[:, 0, :]  # select only heads
          ]

     all_data.append(objects_data)
     break

In [ ]:
plt.imshow(im)